<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/faiss_lsh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/
2. https://www.youtube.com/watch?v=e_SBq3s20M8


In [1]:
!apt install libomp-dev
!pip install --upgrade faiss-cpu faiss-gpu


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 0s (1,480 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

## 介绍

局部敏感哈希 (LSH)是近似最近邻 (ANN) 搜索中广泛使用的技术。高效相似性搜索的解决方案是一种有利可图的解决方案——它是数十亿（甚至数万亿）美元公司的核心。

谷歌、Netflix、亚马逊、Spotify、Uber 等大公司的许多核心功能都依赖相似性搜索。

亚马逊使用相似性搜索来比较客户，根据最相似客户的购买历史找到新产品推荐。

每次使用 Google 时，您都会在查询/搜索词与 Google 索引的互联网之间执行相似性搜索。

如果 Spotify 成功地推荐了好音乐，那是因为他们的相似性搜索算法成功地将您与具有类似良好（或不太好）音乐品味的其他客户匹配。

LSH 是产生高质量搜索的原始技术之一，同时保持闪电般的快速搜索速度。在本文中，我们将介绍该算法背后的理论，以及易于理解的 Python 实现！



## 搜索复杂性
一个包含数百万甚至数十亿个样本的数据集——我们如何有效地比较所有这些样本？

即使在最好的硬件上，比较所有对也是不可能的。这产生的最佳复杂度为 O(n²)。即使将单个查询与数十亿个样本进行比较，我们仍然返回 O(n) 的最佳复杂度。

我们还需要考虑单个相似性计算背后的复杂性——每个样本都存储为一个向量，通常是非常高维的向量——这进一步增加了我们的复杂性。

我们怎样才能避免这种情况呢？是否有可能执行亚线性复杂度的搜索？是的！

解决方案是近似搜索。我们可以近似并将搜索范围限制为最相关的向量，而不是比较每个向量（穷举搜索）。

LSH 是一种为我们提供次线性搜索时间的算法。在本文中，我们将介绍 LSH 并了解其魔法背后的逻辑。

## 局部敏感哈希

当我们考虑[查找相似向量](https://www.pinecone.io/learn/series/nlp/dense-vector-embeddings-nlp/)的复杂性时，我们发现即使数据集相当小，比较所有内容所需的计算量也非常巨大。

让我们考虑一个向量索引。如果我们只引入一个新向量并尝试找到最接近的匹配，我们必须将该向量与数据库中的每个其他向量进行比较。这给了我们一个*线性时间复杂度*——它无法扩展到更大数据集中的快速搜索。

如果我们想要将所有这些向量相互比较，问题就更糟了——实现这一点的最佳排序方法至多是对*数线性时间复杂度*。

所以我们需要一种方法来减少比较次数。理想情况下，我们只想比较我们认为是潜在匹配项或*候选对的*向量。

局部敏感哈希（LSH）允许我们做到这一点。

LSH 由多种不同的方法组成。在本文中，我们将介绍传统方法（由多个步骤组成）、shingling、MinHashing 和最终的banded LSH 函数。

从本质上讲，最终的 LSH 函数允许我们对同一样本进行多次分段和哈希处理。*当我们发现一对向量至少一次*被哈希为相同的值时，我们将它们标记为*候选对*- 即*潜在*匹配。

这是一个与 Python 字典中使用的过程非常相似的过程。我们有一个键值对，我们将其输入字典中。键通过字典哈希函数处理并映射到特定的桶。然后我们将相应的值连接到该存储桶。

![典型的哈希函数旨在将不同的值（无论多么相似）放入不同的桶中。](https://cdn.sanity.io/images/vr8gru94/production/faaae9e55d33b08629574243d8456446650df096-1400x787.png)

典型的哈希函数旨在将不同的值（无论多么相似）放入不同的桶中。

然而，这种类型的哈希函数与 LSH 中使用的哈希函数有一个关键的区别。对于字典，我们的目标是最大限度地减少多个键值映射到同一个存储桶的机会 - 我们最大限度地*减少冲突*。

LSH 几乎相反。在 LSH 中，我们希望*最大化碰撞*——尽管理想情况下仅适用于*相似的*输入。

![LSH 函数旨在将相似的值放入相同的存储桶中。](https://cdn.sanity.io/images/vr8gru94/production/862f88182a796eb16942c47d93ee03ba4cdaee4d-1920x1080.png)

LSH 函数旨在将相似的值放入相同的存储桶中。

LSH 中没有*单一的*散列方法。事实上，它们都共享相同的*“通过哈希函数存储相似样本”*逻辑，但除此之外它们可能有很大差异。

我们已经简要描述并将在本文的其余部分中介绍的方法可以被描述为*传统*方法，使用*shingling*、*MinHashing*和*banding*。

还有其他几种技术，例如我们在另一篇文章中介绍的[随机投影。](https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing-random-projection/)

------

## Shingling、MinHashing 和 LSH

我们正在探索的 LSH 方法由三个步骤组成。*首先，我们使用k-shingling（和 one-hot 编码）*将文本转换为稀疏向量，然后使用*minhashing*创建“签名”——将其传递到我们的 LSH 过程以清除*候选对*。

![我们将在本文中讨论 LSH 流程的高级视图。](https://cdn.sanity.io/images/vr8gru94/production/89413953597fbfdd36c4fa77ca0eeafaf6cf944a-1280x980.png)

我们将在本文中讨论 LSH 流程的高级视图。

我们将在以后的文章中讨论其他一些 LSH 方法。但现在，让我们更深入地了解传统流程。

## k-Shingling
k-Shingling，或简称 shingling — 是将文本字符串转换为一组“shingles”的过程。该过程类似于沿着文本字符串移动长度为 k 的窗口并在每一步拍照。我们整理所有这些图片来创建我们的shinling 集合。

![](https://d33wubrfki0l68.cloudfront.net/80780df0c70ba25ef81de1b0bad918c2a3ed1729/a157b/images/locality-sensitive-hashing-5.mp4)

Shingling还可以删除重复的项目（因此称为“集合”）。我们可以在 Python 中创建一个简单的 k-shingling 函数，如下所示：



In [2]:
import numpy as np

def build_shingles(sentence: str, k: int):
    shingles = []
    for i in range(len(sentence) - k):
        shingles.append(sentence[i:i+k])
    return set(shingles)

def build_vocab(shingle_sets: list):
    # convert list of shingle sets into single set
    full_set = {item for set_ in shingle_sets for item in set_}
    vocab = {}
    for i, shingle in enumerate(list(full_set)):
        vocab[shingle] = i
    return vocab

def one_hot(shingles: set, vocab: dict):
    vec = np.zeros(len(vocab))
    for shingle in shingles:
        idx = vocab[shingle]
        vec[idx] = 1
    return vec

In [53]:
a = "flying fish flew by the space station"
b = "we will not allow you to bring your pet armadillo along"
c = "he figured a few sticks of dynamite were easier than a fishing pole to catch fish"

k = 2
a = build_shingles(a, k)
b = build_shingles(b, k)
c = build_shingles(c, k)
#vocab = a.union(b).union(c)
vocab = build_vocab([a,b,c])

print(a)
print(b)
print(c)
print(len(vocab),vocab)

# one-hot encode our shingles
a_1hot = one_hot(a, vocab)
b_1hot = one_hot(b, vocab)
c_1hot = one_hot(c, vocab)
print(a_1hot)
print(b_1hot)
print(c_1hot)


{'ly', 'h ', 'io', 'e ', 'fl', 'th', 'sh', ' b', 'at', 'ti', 'le', ' f', 'by', 'ac', 'he', 'fi', 'ew', 'sp', ' t', 'st', 'ng', ' s', 'yi', 'in', 'g ', 'ce', 'w ', 'is', 'pa', 'y ', 'ta'}
{'ot', 'di', 'we', 'yo', 'al', 'e ', ' b', ' p', 'ur', ' y', 'il', 't ', ' a', 'r ', 'u ', 'rm', 'to', 'no', ' n', 'ri', 'ou', 'l ', 'lo', 'et', 'ad', 'll', 'o ', ' t', 'on', 'ng', 'in', 'ar', 'pe', 'ma', 'br', 'g ', 'wi', 'w ', 'ow', ' w'}
{'ie', 'ha', 'we', 'ca', 'h ', 'e ', 'er', 'ed', 'th', 'tc', 'sh', ' e', 'at', ' p', 'ti', 'po', 'le', 'ur', ' f', 'hi', 'he', ' a', 'am', 'r ', 'gu', ' o', 's ', 'fi', 'ig', 'ol', 'to', 'ew', 'as', 'n ', 'fe', ' c', 'yn', 're', 'o ', 'd ', ' t', 'dy', 'ks', 'st', 'ng', ' s', 'ck', 'ea', 'mi', 'in', 'te', 'f ', 'of', 'ch', 'g ', 'na', 'w ', 'si', 'is', ' d', 'it', ' w', 'a ', 'an', 'ic'}
102 {'ly': 0, 'ot': 1, 'al': 2, 'er': 3, ' b': 4, ' p': 5, ' y': 6, 'am': 7, 'gu': 8, 'to': 9, 'sp': 10, 'l ': 11, 'as': 12, 'n ': 13, 'll': 14, 're': 15, 'on': 16, ' s': 17, 'ck': 

((3, 102),
 array([[1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
         1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
         1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
         1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1.],
        [0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
         1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
         1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
         0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 1., 1., 

有了这个，我们就有了shingles。接下来，我们创建稀疏向量。为此，我们首先需要合并所有集合以创建一个大集合，其中包含所有集合中的所有shingles - 我们称之为词汇表（或词汇）。

![我们所有的叠瓦集都被合并以创建我们的词汇。](https://cdn.sanity.io/images/vr8gru94/production/3b72d736d96c9da94344668c96a6b9b066522bb6-1280x720.png)

*我们所有的shingled集合合并以创建我们的词汇*。

我们使用这个词汇来创建每个集合的稀疏向量表示。我们所做的就是创建一个充满零且长度与我们的词汇相同的空向量——然后，我们看看哪些shingle出现在我们的集合中。

![为了创建我们的 one-hot 编码，我们的单个 shingle 集与我们的词汇相匹配，这表明我们应该在零向量中放置词汇（我们在代码中使用 shingle-to-index 字典）。](https://cdn.sanity.io/images/vr8gru94/production/c259242606006f5c5505a6e677f3d05be75a26da-1280x720.png)

为了创建我们的 one-hot 编码，我们的单个 shingle 集与我们的词汇相匹配，这表明我们应该在零向量中放置词汇（我们在代码中使用 shingle-to-index 字典）。

对于出现的每个 shingle，我们识别该 shingle 在我们的词汇表中的位置，并将新的零向量中的相应位置设置为 1。可以认为这是*one-hot 编码*。

### 最小哈希

Minhashing 是我们过程的下一步，它允许我们将稀疏向量转换为密集向量。现在，作为一个预警——这部分过程最初可能看起来很混乱——但一旦你掌握了它，它就会非常简单。

我们有稀疏向量，我们所做的就是为签名中的每个位置（例如，密集向量）随机生成一个 minhash 函数。

因此，如果我们想创建一个包含 20 个数字的密集向量/签名，我们将使用 20 个 minhash 函数。

现在，这些 MinHash 函数只是数字的随机顺序 - 我们从*1*计数到最终数字（即 len(vocab)）。由于这些数字的顺序已被随机化，因此我们可能会发现数字 1 位于随机化 MinHash 函数的第57位（例如）

https://d33wubrfki0l68.cloudfront.net/2b77a22ec9933902bb46e2f19b753654fc58d145/91c4c/images/locality-sensitive-hashing-8.mp4

*我们的签名值是通过首先采用随机排列的计数向量（从 1 到 len(vocab)+1）创建的，并找到与稀疏向量中的 1 对齐的最小数字。*

上面，我们使用了包含*六个*值的较小词汇，因此我们可以轻松地可视化该过程。

我们查看稀疏向量并说：“vocab[1] 处的这个shingle是否存在于我们的集合中？”。如果存在，稀疏向量值为 1，在这种情况下，它不*存在*（因此值为 0）。因此，我们转向数字*2*，确定其位置 (0) 并提出相同的问题。这次，答案是*肯定的，*所以我们的 minhash 输出是**2**。

这就是我们在 minhash 签名中生成一个值的方式。但我们需要生成 20 个（或更多）这些值。因此，我们为每个签名位置分配不同的 minhash 函数 - 并重复该过程。

![这里我们使用四个 minhash 函数/向量来创建一个四位签名向量。 如果您在每个 minhash 函数中进行计数（从 1 开始），并确定第一个与稀疏向量中的 1 对齐的值 — 您将得到 2412。](https://cdn.sanity.io/images/vr8gru94/production/866cea917043cfd7eb8221fc1a3b715a61e9d14f-1280x720.png)

这里我们使用四个 minhash 函数/向量来创建一个四位签名向量。如果您在每个 minhash 函数中进行计数（从 1 开始），并确定第一个与稀疏向量中的 1 对齐的值 — 您将得到 2412。

最后，我们生成我们的 minhash 签名 - 或密集向量。

让我们继续用代码编写它。我们分三步走：

1. 生成随机的 MinHash 向量。


In [21]:
hash_ex = list(range(1, len(vocab)+1))
print(hash_ex)  # we haven't shuffled yet

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [22]:
from random import shuffle

shuffle(hash_ex)
print(hash_ex)

[6, 34, 78, 88, 12, 21, 72, 19, 28, 96, 53, 83, 33, 92, 24, 18, 98, 29, 74, 23, 42, 16, 49, 101, 15, 85, 44, 47, 45, 69, 93, 17, 71, 58, 79, 2, 86, 52, 46, 32, 65, 99, 7, 22, 82, 61, 5, 89, 95, 75, 68, 1, 48, 70, 35, 20, 37, 90, 25, 40, 87, 57, 63, 94, 38, 51, 102, 67, 8, 9, 43, 31, 91, 54, 26, 84, 36, 97, 30, 10, 55, 56, 13, 80, 4, 41, 27, 100, 66, 59, 73, 81, 60, 64, 50, 11, 62, 77, 76, 14, 3, 39]


2. 循环遍历这个随机的 MinHash 向量（从 1 开始），并将每个值的索引与稀疏向量 a_1hot 中的等效值进行匹配。如果我们找到对应值—该索引就是我们的签名值。

In [23]:
print(f"7 -> {hash_ex.index(7)}") # note that value 7 can be found at index in hash_ex

7 -> 42


In [24]:
#We now have a randomized list of integers which we can use in creating our *hashed* signatures.
#What we do now is begin counting from `1` through to `len(vocab) + 1`,
#extracting the position of this number in our new `hash_ex` list, like so:

for i in range(1, 5):
    print(f"{i} -> {hash_ex.index(i)}")

1 -> 51
2 -> 35
3 -> 100
4 -> 84


In [32]:
#What we do with this is count up from `1` to `len(vocab) + 1`
#and find if the resultant `hash_ex.index(i)` position
#in our one-hot encoded vectors contains a positive value (`1`) in that position, like so:
for i in range(1, len(vocab)+1):
    idx = hash_ex.index(i)
    signature_val = a_1hot[idx]
    print(f"{i} -> {idx} -> {signature_val}")
    if signature_val == 1:
        print('match!')
        break

1 -> 51 -> 0.0
2 -> 35 -> 0.0
3 -> 100 -> 0.0
4 -> 84 -> 1.0
match!


3. 从1和2的多次迭代构建签名（我们将上面的代码形式化为一些更易于使用的函数）：



In [33]:
def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

# we create 20 minhash vectors
minhash_func = build_minhash_func(len(vocab), 20)

In [34]:
def create_hash(vector: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

In [38]:
# now create signatures
a_sig = create_hash(a_1hot)
b_sig = create_hash(b_1hot)
c_sig = create_hash(c_1hot)

print(a_sig)
print(b_sig)
print(c_sig)

[84, 10, 21, 0, 79, 42, 86, 17, 28, 0, 0, 101, 25, 55, 84, 4, 46, 64, 21, 34]
[79, 50, 21, 91, 50, 14, 20, 24, 50, 11, 5, 50, 68, 69, 19, 4, 46, 64, 94, 95]
[53, 74, 27, 42, 18, 73, 82, 24, 85, 77, 5, 23, 81, 55, 84, 44, 46, 64, 74, 23]


这就是 MinHashing——它实际上没有比这更复杂的了。我们采用了一个稀疏向量并将其压缩为一个更密集的 20 个数字签名。

## 从稀疏到签名的信息传输 (Information Transfer from Sparse to Signature)
信息是否真正保留在我们更大的稀疏向量和更小的稠密向量之间？对我们来说，直观地识别这些新的密集向量中的模式并不容易，但我们可以计算向量之间的相似度。

如果在我们缩小规模的过程中信息确实被保留下来了——向量之间的相似度肯定也会相似吧？

好吧，我们可以测试一下。我们使用 Jaccard 相似度来计算shingle格式的句子之间的相似度- 然后以签名格式重复相同的向量：



In [39]:
def jaccard(a: set, b: set):
    return len(a.intersection(b)) / len(a.union(b))

In [40]:
jaccard(a, b), jaccard(set(a_sig), set(b_sig))

(0.109375, 0.17857142857142858)

In [41]:
jaccard(a, c), jaccard(set(a_sig), set(c_sig))

(0.24675324675324675, 0.1724137931034483)

我们看到两者的相似度分数非常接近——所以信息似乎被保留了。让我们再试一次 b 和 c：



In [42]:
jaccard(b, c), jaccard(set(b_sig), set(c_sig))

(0.15384615384615385, 0.12903225806451613)

在这里，我们发现了更高的相似性，正如我们所期望的那样——看起来我们的稀疏向量和签名之间保留了相似性信息！因此，我们现在已做好充分准备，可以进入 LSH 流程。



## Band and Hash

识别相似句子的最后一步是 LSH 函数本身。

我们将采用 LSH 的Band方法——我们可以将其描述为传统方法。它将获取我们的签名，对每个签名的哈希片段进行哈希处理，并寻找哈希冲突 - 正如我们在本文前面所描述的那样。

![签名构建过程的高级视图。 我们获取文本，构建一个 shingle 集，使用我们的词汇对其进行一次性编码，并通过我们的 minhashing 过程对其进行处理。](https://cdn.sanity.io/images/vr8gru94/production/1a1e12c6e1e7c229319b0a5b96d0300922215205-1280x850.png)

签名构建过程的高级视图。我们获取文本，构建一个 shingle 集，使用我们的词汇对其进行一次性编码，并通过我们的 minhashing 过程对其进行处理。

*通过这种方法，我们生成这些长度相等的向量，其中包含 1 → len(vocab) 范围内的正整数值——这些是我们通常输入到该*LSH算法中的签名。

现在，如果我们要将这些向量中的每一个作为一个整体进行哈希处理，我们可能会很难构建一个哈希函数来准确识别它们之间的相似性——我们不要求整个向量相等，只要求其中的部分相似。

在大多数情况下，即使两个向量的部分可能完全匹配 - 如果向量的其余部分不相等，该函数可能会将它们散列到*单独的*桶中。

我们不想要这个。我们希望具有某些相似性的签名被散列到同一个桶中，从而被识别为候选对。

### 怎么运行的

banding方法通过将向量分成称为bands b的子部分来解决这个问题。然后，我们不是通过哈希函数处理完整的向量，而是通过哈希函数传递向量的每个band。

想象一下我们将 100 维向量分成 20 个Band。这给了我们 20 个机会来识别向量之间的匹配子向量。

![我们将签名分成 b 个子向量，每个子向量都通过哈希函数进行处理（我们可以使用单个哈希函数，或 b 个哈希函数）并映射到哈希桶。](https://cdn.sanity.io/images/vr8gru94/production/1b1fc2e08469ea024573078b275f7228e2e7d824-1920x1080.png)

我们将签名分成 b 个子向量，每个子向量都通过哈希函数进行处理（我们可以使用单个哈希函数，或 b 个哈希函数）并映射到哈希桶。

我们现在可以添加一个更灵活的条件 - 给定任何两个子向量之间的冲突，我们将各自的完整向量视为候选对。

![我们将签名分成子向量。 所有签名中的每个等效子向量必须通过相同的哈希函数进行处理。 然而，没有必要对每个子向量使用不同的哈希函数（我们可以对所有子向量仅使用一个哈希函数）。](https://cdn.sanity.io/images/vr8gru94/production/00a27d5963a54c82b9f751845218b6beb8c09324-1280x720.png)

我们将签名分成子向量。所有签名中的每个等效子向量必须通过相同的哈希函数进行处理。然而，没有必要对每个子向量使用不同的哈希函数（我们可以对所有子向量仅使用一个哈希函数）。

现在，只有两个向量的一部分必须匹配，我们才能考虑它们。但当然，这也会增加误报的数量（我们将不相似的样本标记为候选匹配）。然而，我们确实尝试尽可能地减少这些。

我们可以实现一个简单的版本。首先，我们首先分割签名向量**a**、**b**和**c**：

In [43]:
def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

In [47]:
#Split into 10 bands, creating rows of `2`
band_a = split_vector(a_sig, 10)
band_b = split_vector(b_sig, 10)
band_c = split_vector(c_sig, 10)

print(band_a)
print(band_b)
print(band_c)


[[84, 10], [21, 0], [79, 42], [86, 17], [28, 0], [0, 101], [25, 55], [84, 4], [46, 64], [21, 34]]
[[79, 50], [21, 91], [50, 14], [20, 24], [50, 11], [5, 50], [68, 69], [19, 4], [46, 64], [94, 95]]
[[53, 74], [27, 42], [18, 73], [82, 24], [85, 77], [5, 23], [81, 55], [84, 44], [46, 64], [74, 23]]


然后我们循环遍历列表来识别子向量之间的任何匹配。如果我们找到任何匹配项，我们会将这些向量作为候选对。



In [48]:
for b_rows, c_rows in zip(band_b, band_c):
    if b_rows == c_rows:
        print(f"Candidate pair: {b_rows} == {c_rows}")
        # we only need one band to match
        break

Candidate pair: [46, 64] == [46, 64]


In [49]:
#And let's do the same for **a**.
for a_rows, b_rows in zip(band_a, band_b):
    if a_rows == b_rows:
        print(f"Candidate pair: {a_rows} == {b_rows}")
        # we only need one band to match
        break

Candidate pair: [46, 64] == [46, 64]


In [50]:
for a_rows, c_rows in zip(band_a, band_c):
    if a_rows == c_rows:
        print(f"Candidate pair: {b_rows} == {c_rows}")
        # we only need one band to match
        break

Candidate pair: [46, 64] == [46, 64]


我们发现两个更相似的句子b和 c— 被识别为候选对。三人组中不太相似的一个- 不被确定为候选者。这是一个很好的结果，但如果我们想真正测试 LSH，我们将需要使用更多数据。



## 测试LSH

到目前为止，我们构建的是一个非常低效的实现——如果你想实现 LSH，这肯定不是可行的方法。相反，使用为相似性搜索而构建的库——例如[Faiss](https://www.pinecone.io/learn/series/faiss/)，或托管解决方案（例如 Pinecone）。

但通过这样的代码，如果没有别的事情的话，应该可以清楚地了解 LSH 的工作原理。然而，我们现在将复制更多数据，因此我们将使用 Numpy 重写迄今为止的内容。

该代码将以相同的方式运行 - 您可以在[此笔记本](https://github.com/pinecone-io/examples/blob/master/learn/search/faiss-ebook/locality-sensitive-hashing-traditional/testing_lsh.ipynb)中找到每个函数（以及说明） 。

### 获取数据

首先，我们需要获取数据。[这里](https://github.com/brmson/dataset-sts)有一个很棒的存储库，其中包含为相似性搜索测试构建的多个数据集。我们将从[这里](https://github.com/brmson/dataset-sts/blob/master/data/sts/sick2014/SICK_train.txt)提取一组句子。



In [51]:
import requests
import pandas as pd
import io

url = "https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt"

text = requests.get(url).text

data = pd.read_csv(io.StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [52]:
sentences = data['sentence_A'].tolist()
sentences[:3]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby']

## Shingles
一旦我们有了数据，我们就可以创建 one-hot 编码——这次存储为 NumPy 数组



In [54]:
k = 8  # shingle size

# build shingles
shingles = []
for sentence in sentences:
    shingles.append(build_shingles(sentence, k))

# build vocab
vocab = build_vocab(shingles)

# one-hot encode our shingles
shingles_1hot = []
for shingle_set in shingles:
    shingles_1hot.append(one_hot(shingle_set, vocab))
# stack into single numpy array
shingles_1hot = np.stack(shingles_1hot)
shingles_1hot.shape

(4500, 36466)

In [55]:
shingles_1hot[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
sum(shingles_1hot[0])  # confirm we have 1s

73.0

现在我们有了独热编码。shingles_1hot 数组包含 4500 sparse 向量，其中每个向量的长度为36466。



## 最小散列法
和以前一样，我们将使用 minhashing 将稀疏向量压缩为密集向量“签名”。同样，我们将使用 NumPy 实现，您可以在此处找到完整的代码。



In [57]:
def minhash_arr(vocab: dict, resolution: int):
    length = len(vocab.keys())
    arr = np.zeros((resolution, length))
    for i in range(resolution):
        permutation = np.random.permutation(len(vocab)) + 1
        arr[i, :] = permutation.copy()
    return arr.astype(int)

def get_signature(minhash, vector):
    # get index locations of every 1 value in vector
    idx = np.nonzero(vector)[0].tolist()
    # use index locations to pull only +ve positions in minhash
    shingles = minhash[:, idx]
    # find minimum value in each hash vector
    signature = np.min(shingles, axis=1)
    return signature

In [58]:
arr = minhash_arr(vocab, 100)

signatures = []

for vector in shingles_1hot:
    signatures.append(get_signature(arr, vector))

# merge signatures into single array
signatures = np.stack(signatures)
signatures.shape

(4500, 100)

In [59]:
signatures[0]

array([ 129,  767,  447,  417,   50,  372,   93, 1989,  317,  650,  279,
        173,  351,  995,  222,  766,  778, 1063,  189,  155,  650, 1381,
       1345,  604, 1736,  516,  224,  173, 2673,  240,  206, 1682, 1358,
         62,  893,  766,  448,   56,  603,  231,  264,  335,  261,  137,
        155,  389,  299, 1064,  380,  905,  253,   33, 1290,   64,  172,
        135,  443,   24,  173,  466,  669, 1633,  237,  572,  662,  175,
       1559,  415,  541,  888,  764,   33,  435,   38,  410,  939,   17,
        474, 1099, 1033,  439,   92,  230,  258,  101,  452,  811,  440,
        152,  184,  278,  213,  348,  343,  202,   21,  451,  125,  307,
        442])

我们将稀疏向量从长度36466压缩到长度100的签名。这是一个很大的区别，但正如我们之前所演示的，这种压缩技术很好地保留了相似性信息。



## LSH
最后，到 LSH 部分。我们将在这里再次使用 Python 字典来散列并存储我们的候选对。完整的代码在[这里](https://github.com/weedge/doraemon-nb/blob/main/testing_lsh.ipynb)。

In [60]:
from itertools import combinations

class LSH:
    buckets = []
    counter = 0
    def __init__(self, b):
        self.b = b
        for i in range(b):
            self.buckets.append({})

    def make_subvecs(self, signature):
        l = len(signature)
        assert l % self.b == 0
        r = int(l / self.b)
        # break signature into subvectors
        subvecs = []
        for i in range(0, l, r):
            subvecs.append(signature[i:i+r])
        return np.stack(subvecs)

    def add_hash(self, signature):
        subvecs = self.make_subvecs(signature).astype(str)
        for i, subvec in enumerate(subvecs):
            subvec = ','.join(subvec)
            if subvec not in self.buckets[i].keys():
                self.buckets[i][subvec] = []
            self.buckets[i][subvec].append(self.counter)
        self.counter += 1

    def check_candidates(self):
        candidates = []
        for bucket_band in self.buckets:
            keys = bucket_band.keys()
            for bucket in keys:
                hits = bucket_band[bucket]
                if len(hits) > 1:
                    candidates.extend(combinations(hits, 2))
        return set(candidates)

In [61]:
b = 20

lsh = LSH(b)

for signature in signatures:
    lsh.add_hash(signature)

In [ ]:
lsh.buckets

值得注意的是，我们的 lsh.buckets 变量实际上为每个band包含一个单独的字典 - 我们不会在不同bands之间混合存储桶。

我们在存储桶中看到向量 ID（行号），因此提取候选对所需要做的就是循环所有存储桶并提取对。



In [63]:
candidate_pairs = lsh.check_candidates()
len(candidate_pairs)

6011

In [64]:
list(candidate_pairs)[:5]

[(3877, 3878), (666, 1141), (2185, 2638), (1979, 1981), (1876, 2004)]

识别出候选对后，我们会将相似度计算限制为仅对这些对 - 我们会发现有些将在我们的相似度阈值内，而其他则不会。

这里的目标是限制我们的范围并降低搜索复杂性，同时仍然保持识别对的高精度。

我们可以通过根据实际[余弦（或 Jaccard）相似度](https://www.pinecone.io/learn/semantic-search/)测量候选对分类（1 或 0）来可视化我们的性能。

![显示候选对 (1) 和非候选对 (0) 相对于对签名的余弦相似度的分布的图表。](https://cdn.sanity.io/images/vr8gru94/production/547308d2f04bb52ab733485a0014696a9c7924d0-1280x720.png)

显示候选对 (1) 和非候选对 (0) 相对于对签名的余弦相似度的分布的图表。

现在，这似乎是一种奇怪的可视化我们表现的方式——你是对的，确实如此——但我们确实有理由。

### 优化bands (Optimizing the Bands)

可以优化我们的带值 b 以改变 LSH 函数的相似性阈值。相似度阈值是我们希望 LSH 函数从非候选对切换到候选对的点。

我们将这种概率相似性关系形式化为：

![给定相似性得分 (s)、band数 (b) 和每个band中的行数 (r) 时，一对被识别为候选对的概率 (P)。](https://cdn.sanity.io/images/vr8gru94/production/e1825efa49d55eb681d73326ea268ed4fdd68004-1780x180.png)

给定相似性得分 (s)、band数 (b) 和每个band中的行数 (r) 时，一对被识别为候选对的概率 (P)。

现在，如果我们要可视化当前 b 和 r 值的概率相似关系，我们应该注意到一个模式：

![候选分类（左 y 轴）和根据相似性（计算或归一化余弦相似性）计算的概率 P（右 y 轴）。 这表明我们计算的概率 P 和相似度 s 值表明候选/非候选对的一般分布。 b 和 r 值分别为 20 和 5。](https://cdn.sanity.io/images/vr8gru94/production/b470799575b8e77911bacb8500977afef06d6c85-1280x720.png)

候选分类（左 y 轴）和根据相似性（计算或归一化余弦相似性）计算的概率 P（右 y 轴）。这表明我们计算的概率 P 和相似度 s 值表明候选/非候选对的一般分布。b 和 r 值分别为 20 和 5。

尽管对齐并不完美，但我们可以看到理论计算概率与真实候选对结果之间的相关性。现在，我们可以通过修改 b 向左或向右推动以不同相似度分数返回候选对的概率：

![计算不同 b 值的相似度 s 的概率 P。 请注意，r 将是 len(signature) / b（在本例中 len(signature) == 100）。](https://cdn.sanity.io/images/vr8gru94/production/aace49fa240778e8ecf6e85ad08a2de7f5385566-1280x720.png)

计算不同 b 值的相似度 s 的概率 P。请注意，r 将是 len(signature) / b（在本例中 len(signature) == 100）。

这些是我们计算的概率值。如果我们认为之前的结果 b == 20 需要太高的相似度才能将对计为候选对 - 我们将尝试将相似度阈值向左移动。

看看这张图，b 值 25 看起来足以改变我们的真实结果。那么，让我们可视化使用 b == 25 时的结果：

![b == 25 时的真实结果和模拟结果以蓝色和洋红色显示。 显示我们之前的 LSH 结果（青色）以供比较。 请注意，这创建了更多候选对。](https://cdn.sanity.io/images/vr8gru94/production/5402cb0b40da6b128a53f69fbcbd36a1ff8bdace-1280x720.png)

b == 25 时的真实结果和模拟结果以蓝色和洋红色显示。显示我们之前的 LSH 结果（青色）以供比较。请注意，这创建了更多候选对。

因为我们现在返回更多的候选对，这自然会导致更多的误报——我们为不同的向量返回“候选对”。这是修改 b 不可避免的结果，我们可以将其想象为：

![增加 b（左移）会增加 FP，同时减少 FN。](https://cdn.sanity.io/images/vr8gru94/production/d6b9466efa2e6875ff98f4cce94ae1737e36c53b-1280x720.png)

增加 b（左移）会增加 FP，同时减少 FN。

棒极了！我们从头开始构建了 LSH 流程，甚至设法调整了我们的相似性阈值。

这就是本文关于 LSH 原理的全部内容。我们不仅介绍了 LSH，还介绍了 shingling 和 MinHash 函数！

在实践中，我们很可能希望使用专门为相似性搜索构建的库来实现 LSH。我们将更详细地介绍 LSH（特别是随机投影方法）及其在 Faiss 中的实现。

但是，如果您希望快速了解相似性搜索中的一些关键索引（及其实现），我们将在[向量索引概述](https://www.pinecone.io/learn/series/faiss/vector-indexes/)中涵盖所有这些索引。

## Resources

- [Jupyter Notebooks](https://github.com/pinecone-io/examples/tree/master/learn/search/faiss-ebook/locality-sensitive-hashing-traditional)

- J. Ullman et al., [Mining of Massive Datasets](http://mmds.org/#ver30)

  